In [1]:
import os
import xml.etree.ElementTree as ET

label_map = {
    "gland": 0
}

def parse_polygon_points(points_str):
    """
    Parsuje ciąg punktów w formacie "x1,y1;x2,y2;...;xn,yn" 
    i zwraca listę par (x, y) jako liczby float.
    """
    points = []
    for point in points_str.split(';'):
        if point.strip():
            x_str, y_str = point.split(',')
            points.append((float(x_str), float(y_str)))
    return points

def polygon_to_bbox(points):
    """
    Na podstawie listy punktów oblicza prostokąt ograniczający (bounding box).
    Zwracane wartości: xtl, ytl, xbr, ybr (lewy górny i prawy dolny róg).
    """
    xs = [p[0] for p in points]
    ys = [p[1] for p in points]
    return min(xs), min(ys), max(xs), max(ys)

def convert_annotation(xml_file, images_dir, labels_dir):
    """
    Konwertuje adnotacje CVAT zapisane w pliku XML, gdzie adnotacje są określone poprzez
    elementy <polygon>, do formatu YOLO.
    
    Parametry:
      xml_file   - ścieżka do pliku anotations.xml
      images_dir - folder, w którym znajdują się obrazy (używany do odczytania wymiarów, jeśli potrzeba)
      labels_dir - folder, do którego zapisywane będą pliki TXT z adnotacjami YOLO
    """
    if not os.path.exists(labels_dir):
        os.makedirs(labels_dir)
    
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    for image in root.findall('image'):
        image_name = image.attrib['name']
        image_width = float(image.attrib['width'])
        image_height = float(image.attrib['height'])

        txt_file_name = os.path.splitext(image_name)[0] + ".txt"
        txt_file_path = os.path.join(labels_dir, txt_file_name)
        
        yolo_lines = []
        
        for poly in image.findall('polygon'):
            label = poly.attrib.get('label')
            if label not in label_map:
                print(f"Ostrzeżenie: etykieta '{label}' nie jest zmapowana. Pomijam ten obiekt.")
                continue

            class_id = label_map[label]
            points_str = poly.attrib.get('points')
            
            points = parse_polygon_points(points_str)
            xtl, ytl, xbr, ybr = polygon_to_bbox(points)
            
            x_center = (xtl + xbr) / 2.0
            y_center = (ytl + ybr) / 2.0
            box_width = xbr - xtl
            box_height = ybr - ytl

            x_center_norm = x_center / image_width
            y_center_norm = y_center / image_height
            width_norm = box_width / image_width
            height_norm = box_height / image_height
            
            yolo_line = f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}"
            yolo_lines.append(yolo_line)
        
        for box in image.findall('box'):
            label = box.attrib.get('label')
            if label not in label_map:
                print(f"Ostrzeżenie: etykieta '{label}' nie jest zmapowana. Pomijam ten obiekt.")
                continue

            class_id = label_map[label]
            xtl = float(box.attrib['xtl'])
            ytl = float(box.attrib['ytl'])
            xbr = float(box.attrib['xbr'])
            ybr = float(box.attrib['ybr'])
            
            x_center = (xtl + xbr) / 2.0
            y_center = (ytl + ybr) / 2.0
            box_width = xbr - xtl
            box_height = ybr - ytl

            x_center_norm = x_center / image_width
            y_center_norm = y_center / image_height
            width_norm = box_width / image_width
            height_norm = box_height / image_height

            yolo_line = f"{class_id} {x_center_norm:.6f} {y_center_norm:.6f} {width_norm:.6f} {height_norm:.6f}"
            yolo_lines.append(yolo_line)

        with open(txt_file_path, "w") as f:
            f.write("\n".join(yolo_lines))
        
        print(f"Przetworzono: {image_name} -> {txt_file_name}")



In [2]:

xml_path = "./LearnSet145/annotations.xml" 
photos_folder = "./LearnSet145/photos"      
labels_folder = "./LearnSet145/yololabbels"
    
convert_annotation(xml_path, photos_folder, labels_folder)


Przetworzono: tile_12544_51968.tif -> tile_12544_51968.txt
Przetworzono: tile_16128_50176.tif -> tile_16128_50176.txt
Przetworzono: tile_19712_19712.tif -> tile_19712_19712.txt
Przetworzono: tile_23296_53760.tif -> tile_23296_53760.txt
Przetworzono: tile_7168_94976.tif -> tile_7168_94976.txt
Przetworzono: tile_8960_84224.tif -> tile_8960_84224.txt
